In [109]:
with open('24.txt', 'r') as f:
    data = f.read()

In [101]:
data = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
"""

In [7]:
def parse(data):
    coords = []
    for line in data.strip().split('\n'):
        coord = []
        i = 0
        while i < len(line):
            if line[i] in "ns":
                coord.append(line[i:i+2])
                i += 2
            else:
                coord.append(line[i])
                i += 1
        coords.append(coord)
    return coords

In [20]:
from collections import defaultdict

In [31]:
def part1(cards):
    base = {
        'e': np.array([+1,-1,0]),
        'ne': np.array([+1,0,-1]),
        'nw': np.array([0,+1,-1])
    }
    base['w'] = -base['e']
    base['se'] = -base['nw']
    base['sw'] = -base['ne']
    counts = defaultdict(int)
    for cs in cards:
        acc = np.array([0,0,0])
        for c in cs:
            acc += base[c]
        counts[tuple(acc)] += 1
    print(counts)
    return sum(1 for count in counts.values() if count & 1 == 1)

In [32]:
part1(parse(data))

defaultdict(<class 'int'>, {(-3, 1, 2): 1, (1, 2, -3): 2, (-3, 0, 3): 1, (2, 0, -2): 2, (1, 1, -2): 2, (-1, 1, 0): 2, (-2, 2, 0): 1, (0, 1, -1): 1, (-2, 1, 1): 1, (0, 2, -2): 2, (3, 0, -3): 1, (0, -2, 2): 1, (0, 0, 0): 1, (2, -2, 0): 1, (-1, 2, -1): 1})


10

In [58]:
from numba import njit, jit

In [61]:
def conway(cards, steps):
    base = {
        'e': np.array([+1,-1,0]),
        'ne': np.array([+1,0,-1]),
        'nw': np.array([0,+1,-1])
    }
    base['w'] = -base['e']
    base['se'] = -base['nw']
    base['sw'] = -base['ne']
    counts = defaultdict(int)
    for cs in cards:
        acc = np.array([0,0,0])
        for c in cs:
            acc += base[c]
        counts[tuple(acc)] += 1
    coords = np.array([np.array(cs) for cs, count in counts.items() if count & 1 ==1])
    min_dim = coords.min(axis=0) - 2
    max_dim = coords.max(axis=0) + 2
    X, Y, _ = max_dim - min_dim
    state = np.zeros((X,Y))
    ox, oy, _ = min_dim
    for x, y, _ in coords:
        state[x-ox,y-oy] = True
    ds = [(0,+1),(0,-1),(+1,0),(-1,0),(+1,-1),(-1,+1)]
    for i in range(steps):
        print(i, state.sum())
        state = np.pad(state, [(1,1),(1,1)], mode='constant', constant_values=False)
        new_state = state.copy()
        X, Y = state.shape
        for x in range(1,X-1):
            for y in range(1,Y-1):
                neighs = sum([state[x+dx,y+dy] for dx, dy in ds])
                if state[x,y] and (neighs == 0 or neighs > 2):
                    new_state[x,y] = False
                elif (not state[x,y]) and neighs == 2:
                    new_state[x,y] = True
        state = new_state
    return state.sum()

In [64]:
set([1,2,3]) | set([4,5])

{1, 2, 3, 4, 5}

In [112]:
conway(parse(data), 100)

4242

In [119]:
def conway(cards, steps):
    base = {
        'e': 1-1j,
        'ne': 1,
        'nw': 1j
    }
    base['w'] = -base['e']
    base['se'] = -base['nw']
    base['sw'] = -base['ne']
    counts = defaultdict(int)
    for cs in cards:
        acc = 0
        for c in cs:
            acc += base[c]
        counts[acc] += 1
    ds = [1,-1,1j,-1j,1-1j,-1+1j]
    blacks = set(cs for cs, count in counts.items() if count & 1 == 1)
    for i in range(steps):
        new_blacks = set()
        whites = defaultdict(int)
        for x in blacks:
            neighs = 0
            for d in ds:
                whites[x+d] += 1
                neighs += (x+d) in blacks
            if neighs == 1 or neighs == 2:
                new_blacks.add(x)
        new_blacks.update([w for w,c in whites.items() if c==2])
        blacks = new_blacks
        
    return len(blacks)

In [120]:
conway(parse(data), 100)

4242